In [2]:
import pandas as pd
raw_data = pd.read_csv('/Users/Desktop/hot100_all_1960_2022.csv')

In [3]:
raw_data.head()

,date,title,artist,rank,last_week,peak,weeks
0,2022-04-02,Heat Waves,Glass Animals,1,1,1,62
1,2022-04-02,Stay,The Kid LAROI & Justin Bieber,2,2,1,37
2,2022-04-02,Super Gremlin,Kodak Black,3,3,3,20
3,2022-04-02,abcdefu,GAYLE,4,4,3,18
4,2022-04-02,Ghost,Justin Bieber,5,6,5,26


In [4]:
date_to_year = lambda x: x.split('-')[0]
raw_data['year'] = raw_data['date'].apply(date_to_year)

In [5]:
hotest = pd.DataFrame(columns = raw_data.columns)
data_group = raw_data.groupby('year')
for year in data_group:
    this_year = year[1].sort_values('weeks', ascending = False)
    this_year = this_year[['title', 'artist', 'weeks']]
    this_year.drop_duplicates(['title', 'artist'], inplace = True)
    this_year_index = this_year.index[0:50]
    hotest = hotest.append(raw_data.iloc[this_year_index])

In [6]:
hotest = hotest.reset_index(drop = True)

In [7]:
hotest.head()

,date,title,artist,rank,last_week,peak,weeks,year
0,1960-04-18,Running Bear,Johnny Preston,91,81,1,27,1960
1,1960-11-07,Finger Poppin' Time,Hank Ballard And The Midnighters,69,57,7,26,1960
2,1960-02-15,Mack The Knife,Bobby Darin,76,59,1,26,1960
3,1960-05-30,Sweet Nothin's,Brenda Lee,63,32,4,24,1960
4,1960-07-11,Sixteen Reasons,Connie Stevens,61,40,3,24,1960


In [9]:
import os
import json
from tqdm import tqdm
from datetime import datetime

In [15]:
print(datetime.now())
token = 'BQBaQMxxUFjfJggKIaXb9hE65H7py8ehit1P-O1tbVu6hdO_c1XKO8gMn48xBV8RrOpCPe--PheGTw4kJuL59eFaKIwKuXDR2oFZWEBSLYV187q3XK7ZODBAu9p1DTki9sOGXBjDGVfITfgpnsOkv43ZEcwv0mMneoQ'

2022-04-03 22:06:48.726355


In [18]:
interval_left = 0
interval_right = interval_left + 3150

In [19]:
audio_feature_dict = \
{'danceability': [],
'energy': [],
'key': [],
'loudness': [],
'mode': [],
'speechiness': [],
'acousticness': [],
'instrumentalness': [],
'liveness': [],
'valence': [],
'tempo': [],
'duration_ms': []}

genres = []

for index in tqdm(hotest.index[interval_left: interval_right]):
    try:
        this_title = hotest.iloc[index]['title'].split()
        this_artist = hotest.iloc[index]['artist'].split()
        if len(this_title) > 1:
            title_query = this_title[0] + '%20' + this_title[1]
        else:
            title_query = this_title[0]
        if len(this_artist) > 1:
            artist_query = this_artist[0] + '%20' + this_artist[1]
        else:
            artist_query = this_artist[0]

        id_query_url = 'https://api.spotify.com/v1/search?q=' + title_query + '&artist=' + artist_query + '&type=track&limit=1'

        id_query_cmd = "curl --request GET \
        --url '" + id_query_url + "' \
        --header 'Authorization: Bearer '" + token + " \
        --header 'Content-Type: application/json'"

        id_query_response = os.popen(id_query_cmd).read()
        id_json = json.loads(id_query_response)
        track_id = id_json['tracks']['items'][0]['id']
        artist_id = id_json['tracks']['items'][0]['artists'][0]['id']
        
        audio_feature_query_cmd = "curl --request GET \
        --url https://api.spotify.com/v1/audio-features/" + track_id + " \
        --header 'Authorization: Bearer '" + token + " \
        --header 'Content-Type: application/json'"
        
        audio_features_response = os.popen(audio_feature_query_cmd).read()
        feature_json = json.loads(audio_features_response)
        
        for key in audio_feature_dict.keys():
            audio_feature_dict[key].append(feature_json[key])
            
        artist_query_cmd = "curl --request GET \
        --url https://api.spotify.com/v1/artists/" + artist_id + " \
        --header 'Authorization: Bearer '" + token + " \
        --header 'Content-Type: application/json'"

        artist_response = os.popen(artist_query_cmd).read()
        artist_json = json.loads(artist_response)
        
        genres.append(artist_json['genres'])
              
    except:
        for key in audio_feature_dict.keys():
            audio_feature_dict[key].append(None)
        genres.append([])

audio_feature_dict['genres'] = genres
with open('./data/' + str(interval_left) + '_' + str(interval_right) + '.json', 'w') as fp:
    json.dump(audio_feature_dict, fp)

100%|██████████| 150/150 [07:09<00:00,  2.86s/it]


In [18]:
final_feature_dict = \
{'danceability': [],
'energy': [],
'key': [],
'loudness': [],
'mode': [],
'speechiness': [],
'acousticness': [],
'instrumentalness': [],
'liveness': [],
'valence': [],
'tempo': [],
'duration_ms': [],
'genres': []}

files = list(os.listdir('/Users/Desktop/json_data/'))
files.sort()
for file in files:
    try:
        file = open('/Users/Desktop/json_data/' + file)
        json_file = json.load(file)
        this_dict = dict(json_file)
        for key in final_feature_dict.keys():
            for index in range(len(this_dict[key])):
                final_feature_dict[key].append(this_dict[key][index])
    except:
        pass

In [19]:
for key in final_feature_dict.keys():
    hotest[key] = final_feature_dict[key]

In [37]:
hotest.drop('rank', axis = 1, inplace = True)
hotest.drop('last_week', axis = 1, inplace = True)

In [20]:
hotest.head()

,date,title,artist,rank,last_week,peak,weeks,year,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genres
0,1960-04-18,Running Bear,Johnny Preston,91,81,1,27,1960,0.610,0.688,...,-9.308,1.0,0.0357,0.573,0.000000,0.164,0.761,119.831,165893.0,[nashville sound]
1,1960-11-07,Finger Poppin' Time,Hank Ballard And The Midnighters,69,57,7,26,1960,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,1960-02-15,Mack The Knife,Bobby Darin,76,59,1,26,1960,0.549,0.529,...,-12.291,0.0,0.1080,0.760,0.000000,0.206,0.464,82.755,184333.0,"[adult standards, brill building pop, easy lis..."
3,1960-05-30,Sweet Nothin's,Brenda Lee,63,32,4,24,1960,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,1960-07-11,Sixteen Reasons,Connie Stevens,61,40,3,24,1960,0.339,0.406,...,-8.955,1.0,0.0319,0.865,0.000011,0.110,0.619,109.783,118133.0,[deep adult standards]


In [21]:
hotest.to_csv('/Users/Desktop/hottest_1960_2022.csv', index = None, encoding = 'utf-8-sig')